In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.FnHs02lqj4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Jy7xp1T4He/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.mU3WKAqKjb/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
import json
from selenium.webdriver.common.by import By

첫 번째 데이터: 역대 박스오피스- 셀레니움 이용한 크롤링

In [ ]:
url = f'https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do'
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome("chromedriver", options=options)
driver.get(url)

# table 찾기
table = driver.find_element('tag name', 'table')

#컬럼값 추출
th_elements = table.find_elements('tag name','th')
th_values = []
for th in th_elements:
    th_values.append(th.text)

#각 행 가져오기
tbody = table.find_element('tag name', 'tbody')
tr_elements = tbody.find_elements('tag name', 'tr') 
td_values = []
for tr in tr_elements:
    td_elements = tr.find_elements('tag name', 'td')
    row_data = [td.text for td in td_elements]
    td_values.append(row_data)

rank_df = pd.DataFrame(td_values, columns=th_values)
rank_df['개봉일'] = pd.to_datetime(rank_df['개봉일'], format='%Y-%m-%d').dt.strftime('%Y%m%d')
driver.quit()

In [ ]:
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      200 non-null    object
 1   영화명     200 non-null    object
 2   개봉일     200 non-null    object
 3   매출액     200 non-null    object
 4   관객수     200 non-null    object
 5   스크린수    200 non-null    object
 6   상영횟수    200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [ ]:
rank_df.head()

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수
0,1,명량,20140730,"135,758,658,810","17,615,919","1,587","188,726"
1,2,극한직업,20190123,"139,655,543,516","16,266,338","2,003","292,816"
2,3,신과함께-죄와 벌,20171220,"115,727,528,087","14,414,658","1,912","214,631"
3,4,국제시장,20141217,"110,947,621,230","14,264,478","1,044","212,705"
4,5,어벤져스: 엔드게임,20190424,"122,492,181,020","13,977,602","2,835","246,433"


In [ ]:
rank_df.tail()

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수
195,196,코코,20180111,"27,766,173,140","3,516,880","1,177","86,072"
196,197,해리 포터와 불의 잔,20051201,"22,097,731,900","3,499,257",504,"45,555"
197,198,비긴 어게인,20140813,"27,433,947,031","3,479,444",525,"87,565"
198,199,태풍,20051214,"22,209,675,300","3,471,150",481,"47,155"
199,200,킹콩,20051214,"21,476,688,172","3,470,059",319,"38,456"


두번째 데이터: 앞에서 크롤링한 역대 박스오피스의 영화명으로 상세 정보 조회<br>
OpenApi 이용

In [ ]:
# movieNames = rank_df['영화명']

#결과값을 저장하기 위한 변수 생성.
default_columns = ['영화코드', '영화명', '영화명(영어)', '제작년도', '개봉일', '영화유형', '제작상태', '제작국가', '영화장르', '대표국가', '대표 장르', '디렉터', '제작사']
df_filtered = pd.DataFrame(columns=default_columns)
cnt = 0

for index, row in rank_df.iterrows():
    cnt=0
    name = row['영화명']

    if name == '좋은 놈, 나쁜 놈, 이상한 놈':
      name = '좋은 놈, 나쁜 놈, 이상한 놈 '
    
    open_date = row['개봉일']
    url = f"http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=f5eef3421c602c6cb7ea224104795888&movieNm={name}&itemPerPage=100"
    if name == '2012':
      url = "https://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=f5eef3421c602c6cb7ea224104795888&movieNm=2012&itemPerPage=100&curPage=4"
    
    r = requests.get(url)
    data = r.json()
    movie_list = data['movieListResult']['movieList']
    
    filtered_movies = []
    none_movies=[]
    if cnt == 0 :
      for movie in movie_list:
        if movie['movieNm'] == name:
          if movie['openDt'] == open_date:
              filtered_movies.append(movie)
              cnt = 1
              break
          elif movie['openDt'] is None or movie['openDt'] == "":
              none_movies.append(movie)
    if cnt == 0 :
      for movie in none_movies:
        if movie['movieNm'] == name:
          filtered_movies.append(movie)
          break

    for movie in filtered_movies:
        movie_data = {
            '영화코드': movie['movieCd'],
            '영화명': movie['movieNm'],
            '영화명(영어)': movie['movieNmEn'],
            '제작년도': movie['prdtYear'],
            '개봉일': movie['openDt'],
            '영화유형': movie['typeNm'],
            '제작상태': movie['prdtStatNm'],
            '제작국가': movie['nationAlt'],
            '영화장르': movie['genreAlt'],
            '대표국가': movie['repNationNm'],
            '대표 장르': movie['repGenreNm'],
            '디렉터': movie['directors'],
            '제작사': movie['companys']
        }
        df_filtered = pd.concat([df_filtered, pd.DataFrame([movie_data])], ignore_index=True)

df_filtered['영화명'] = df_filtered['영화명'].replace('좋은 놈, 나쁜 놈, 이상한 놈 ', '좋은 놈, 나쁜 놈, 이상한 놈')



In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화코드     200 non-null    object
 1   영화명      200 non-null    object
 2   영화명(영어)  200 non-null    object
 3   제작년도     200 non-null    object
 4   개봉일      200 non-null    object
 5   영화유형     200 non-null    object
 6   제작상태     200 non-null    object
 7   제작국가     200 non-null    object
 8   영화장르     200 non-null    object
 9   대표국가     200 non-null    object
 10  대표 장르    200 non-null    object
 11  디렉터      200 non-null    object
 12  제작사      200 non-null    object
dtypes: object(13)
memory usage: 20.4+ KB


In [ ]:
# 디렉터 이름만 가져오기
df_filtered['디렉터'] = df_filtered['디렉터'].apply(lambda x: x[0]['peopleNm'] if x else '')

# 제작사 이름만 가져오기
df_filtered['제작사'] = df_filtered['제작사'].apply(lambda x: x[0]['companyNm'] if x else '')

info_df = df_filtered

수집한 2개의 데이터<br>
1. 2004년 이후 국내 개봉한 영화 기준 역대 박스오피스 200위 까지 데이터 --> rank_df
2. 해당 영화에 대한 상세 정보가 담긴 데이터 --> info_df

In [ ]:
rank_df.head()

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수
0,1,명량,20140730,"135,758,658,810","17,615,919","1,587","188,726"
1,2,극한직업,20190123,"139,655,543,516","16,266,338","2,003","292,816"
2,3,신과함께-죄와 벌,20171220,"115,727,528,087","14,414,658","1,912","214,631"
3,4,국제시장,20141217,"110,947,621,230","14,264,478","1,044","212,705"
4,5,어벤져스: 엔드게임,20190424,"122,492,181,020","13,977,602","2,835","246,433"


In [ ]:
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      200 non-null    object
 1   영화명     200 non-null    object
 2   개봉일     200 non-null    object
 3   매출액     200 non-null    object
 4   관객수     200 non-null    object
 5   스크린수    200 non-null    object
 6   상영횟수    200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [ ]:
info_df.head()

,영화코드,영화명,영화명(영어),제작년도,개봉일,영화유형,제작상태,제작국가,영화장르,대표국가,대표 장르,디렉터,제작사
0,20129370,명량,The Admiral: Roaring Currents,2013,20140730,장편,개봉,한국,"사극,액션",한국,사극,김한민,(주)빅스톤픽쳐스
1,20182530,극한직업,Extreme Job,2018,20190123,장편,개봉,한국,코미디,한국,코미디,이병헌,(주)어바웃잇
2,20150976,신과함께-죄와 벌,Along with the Gods: The Two Worlds,2017,20171220,장편,개봉,한국,"판타지,드라마",한국,판타지,김용화,리얼라이즈픽쳐스(주)
3,20137048,국제시장,Ode to My Father,2014,20141217,장편,개봉,한국,드라마,한국,드라마,윤제균,(주)제이케이필름
4,20184889,어벤져스: 엔드게임,Avengers: Endgame,2018,20190424,장편,개봉,미국,"액션,SF",미국,액션,안소니 루소,


In [ ]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화코드     200 non-null    object
 1   영화명      200 non-null    object
 2   영화명(영어)  200 non-null    object
 3   제작년도     200 non-null    object
 4   개봉일      200 non-null    object
 5   영화유형     200 non-null    object
 6   제작상태     200 non-null    object
 7   제작국가     200 non-null    object
 8   영화장르     200 non-null    object
 9   대표국가     200 non-null    object
 10  대표 장르    200 non-null    object
 11  디렉터      200 non-null    object
 12  제작사      200 non-null    object
dtypes: object(13)
memory usage: 20.4+ KB


In [ ]:
df_rank_info = pd.merge(rank_df, info_df, on=['영화명', '개봉일'])

In [ ]:
movies_name = df_rank_info['영화명']
score_df = pd.DataFrame(columns=['영화명', '평점'])
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome("chromedriver", options=options)

for name in movies_name:
  url = f'https://search.naver.com/search.naver?query=영화 {name}'
  driver.get(url)
  print(name)
  try:
    div_1 = driver.find_element(By.CLASS_NAME, 'detail_info')
    div_2 = div_1.find_element(By.TAG_NAME, 'dl')
    divs = div_2.find_elements(By.TAG_NAME, 'div')
    div_3 = divs[2]
    score = div_3.find_element(By.TAG_NAME, 'dd').text
    score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)
  except:
    score_df = score_df.append({'영화명': name, '평점': 0}, ignore_index=True)
  

print(score_df)


명량


<ipython-input-40-e09cba8b4198>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)


극한직업


<ipython-input-40-e09cba8b4198>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)


신과함께-죄와 벌


...

<ipython-input-40-e09cba8b4198>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)


비긴 어게인


<ipython-input-40-e09cba8b4198>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)


태풍
             영화명    평점
0             명량  8.88
1           극한직업  9.20
2      신과함께-죄와 벌  8.73
3           국제시장  9.16
4     어벤져스: 엔드게임  9.50
..           ...   ...
195       신의 한 수  8.66
196           코코  9.20
197  해리 포터와 불의 잔  9.41
198       비긴 어게인  9.13
199           태풍  6.13

[200 rows x 2 columns]


<ipython-input-40-e09cba8b4198>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({'영화명': name, '평점': score}, ignore_index=True)


In [ ]:
result_df = pd.merge(df_rank_info, score_df, on=['영화명'])

In [ ]:
result_df.loc[result_df['평점'] == '2017.01.04.', '평점'] = '0'
result_df.loc[result_df['평점'] == '0', '평점'] = pd.NA

In [ ]:
result_df.loc[result_df['평점'] == 0, '평점'] = '0'
result_df.loc[result_df['평점'] == '0', '평점'] = pd.NA

In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   순위       200 non-null    object
 1   영화명      200 non-null    object
 2   개봉일      200 non-null    object
 3   매출액      200 non-null    object
 4   관객수      200 non-null    object
 5   스크린수     200 non-null    object
 6   상영횟수     200 non-null    object
 7   영화코드     200 non-null    object
 8   영화명(영어)  200 non-null    object
 9   제작년도     200 non-null    object
 10  영화유형     200 non-null    object
 11  제작상태     200 non-null    object
 12  제작국가     200 non-null    object
 13  영화장르     200 non-null    object
 14  대표국가     200 non-null    object
 15  대표 장르    200 non-null    object
 16  디렉터      200 non-null    object
 17  제작사      200 non-null    object
 18  평점       179 non-null    object
dtypes: object(19)
memory usage: 31.2+ KB


In [ ]:
result_df['평점'].loc[150:200]

150    8.76
151    8.53
152    <NA>
153    7.69
154    8.02
155    9.65
156    9.24
157    8.06
158    8.27
159    8.68
160    <NA>
161    5.65
162    <NA>
163    <NA>
164    8.28
165    8.57
166    8.32
167    7.76
168    8.43
169    8.78
170    8.83
171    8.87
172    <NA>
173    8.12
174    8.24
175    8.07
176    <NA>
177    <NA>
178    8.17
179    7.16
180    8.58
181    9.09
182    8.91
183    <NA>
184    <NA>
185    8.28
186    8.63
187    7.49
188    9.12
189    9.18
190    <NA>
191    8.66
192    9.04
193    <NA>
194    9.06
195    8.66
196    9.20
197    9.41
198    9.13
199    6.13
Name: 평점, dtype: object

In [ ]:
result_df.to_csv('movie_data.csv', encoding='utf-8')